# Listas (arreglos dinámicos)

**Curso:** Estructuras de Datos (Ingeniería de Sistemas)

**Propósito:** Comprender y aplicar las ideas clave del tema, analizando complejidad y usos prácticos.

### Objetivos de aprendizaje
- Reconocer el modelo de datos y operaciones fundamentales.
- Analizar la complejidad temporal y espacial de las operaciones clave.
- Implementar y probar funciones en Python con casos de uso reales.
- Comparar ventajas y limitaciones frente a alternativas.


### Teoría esencial

Las listas en Python son contenedores ordenados y mutables. En estructuras de datos,
representan arreglos dinámicos sobre un bloque contiguo de memoria con realocación amortizada.
Operaciones típicas:
- Acceso por índice: O(1)
- Inserción/eliminación al final (append/pop): O(1) amortizado
- Inserción/eliminación en posición arbitraria: O(n)
- Búsqueda lineal: O(n)

### Guía de estudio
- Identifica operaciones fundamentales y su costo asintótico.
- Observa invariante(s) de la estructura (lo que siempre debe cumplirse).
- Relaciona la estructura con patrones de uso en software (colas de trabajo, caches, planificadores, etc.).

### Implementación base (desde el archivo `.py`)
El siguiente bloque contiene el código original.

In [ ]:
print('*** Listas y Diccionarios ***')

personas = [
    {
        'nombre': 'Regina',
        'apellido': 'Flores',
        'edad': 21
    },
    {
        'nombre': 'Alejandro',
        'apellido': 'Reyes',
        'edad': 32
    }
]

print(personas)

# Acceder a un diccionario desde una lista
print(f'''Detalle del primer elemento de la lista:
    Nombre: {personas[0].get('nombre')}
    Apellido: {personas[0].get('apellido')}
    Edad: {personas[0].get('edad')}''')

# Recorrer los elementos de la lista
print()
for contador, persona in enumerate(personas):
    print(f'{contador} - Persona: {persona}')
    #print(f'Detalle: Nombre: {persona.get('nombre')}, Apellido: {persona.get('apellido')}')

In [1]:
from typing import List, Dict, Any, Callable, Iterable, Optional, Tuple
from collections import defaultdict
import timeit
import random
import statistics

# -------------------------------------------------------------
# 1. Indexar por campo
# -------------------------------------------------------------
def indexar_por_campo(registros: List[Dict[str, Any]], campo: str) -> Dict[Any, Dict[str, Any]]:
    """Construye un índice (dict) para acceso O(1) por valor de campo.
    Si hay duplicados, conserva el último (patrón similar a sobrescritura).
    """
    idx = {}
    for rec in registros:
        if campo in rec:
            idx[rec[campo]] = rec
    return idx

# -------------------------------------------------------------
# 2. Agrupar por campo
# -------------------------------------------------------------
def agrupar_por_campo(registros: List[Dict[str, Any]], campo: str) -> Dict[Any, List[Dict[str, Any]]]:
    """Agrupa los registros en listas por el valor del campo dado."""
    grupos = defaultdict(list)
    for rec in registros:
        grupos[rec.get(campo)].append(rec)
    return dict(grupos)

# -------------------------------------------------------------
# 3. Detectar duplicados y deduplicar
# -------------------------------------------------------------
def detectar_duplicados(registros: List[Dict[str, Any]], campo: str) -> Dict[Any, int]:
    conteo = defaultdict(int)
    for rec in registros:
        if campo in rec:
            conteo[rec[campo]] += 1
    return {k: v for k, v in conteo.items() if v > 1}


def deduplicar_por_campo(registros: List[Dict[str, Any]], campo: str, keep: str = "last") -> List[Dict[str, Any]]:
    """Elimina duplicados por valor de campo.
    keep: 'first' o 'last'.
    """
    vistos = {}
    for rec in registros:
        if campo not in rec:
            continue
        key = rec[campo]
        if keep == 'first' and key in vistos:
            continue  # conservar existente
        vistos[key] = rec  # sobrescribe si keep == 'last'
    return list(vistos.values())

# -------------------------------------------------------------
# 4. Combinar (join) dos listas de diccionarios
# -------------------------------------------------------------
class JoinModo:
    INNER = "inner"
    LEFT = "left"
    RIGHT = "right"
    OUTER = "outer"


def combinar_listas_por_campo(
    left: List[Dict[str, Any]],
    right: List[Dict[str, Any]],
    key: str,
    modo: str = JoinModo.INNER,
    sufijos: Tuple[str, str] = ("_left", "_right")
) -> List[Dict[str, Any]]:
    """Combina dos listas por una clave compartida (similar a joins SQL).
    Devuelve lista de diccionarios fusionados.
    """
    idx_right = indexar_por_campo(right, key)
    resultado = []
    usados_right = set()

    for rec_left in left:
        k = rec_left.get(key)
        rec_right = idx_right.get(k)
        if rec_right is not None:  # match
            fusion = {**rec_left}
            for rk, rv in rec_right.items():
                if rk == key:
                    continue
                if rk in fusion:
                    fusion[rk + sufijos[1]] = rv
                else:
                    fusion[rk] = rv
            resultado.append(fusion)
            usados_right.add(k)
        else:
            if modo in (JoinModo.LEFT, JoinModo.OUTER):
                # Aportar rec_left con campos right como None
                fusion = {**rec_left}
                # Podríamos opcionalmente explorar claves típicas de right para anexar None
                resultado.append(fusion)

    if modo in (JoinModo.RIGHT, JoinModo.OUTER):
        # Agregar los de right que no hicieron match
        idx_left = indexar_por_campo(left, key)
        for rec_right in right:
            k = rec_right.get(key)
            if k not in idx_left:
                fusion = {key: k}
                for rk, rv in rec_right.items():
                    if rk == key:
                        continue
                    fusion[rk] = rv
                resultado.append(fusion)

    return resultado

# -------------------------------------------------------------
# 5. Filtrado declarativo
# -------------------------------------------------------------
def filtrar_por_predicado(registros: List[Dict[str, Any]], predicado: Callable[[Dict[str, Any]], bool]) -> List[Dict[str, Any]]:
    return [r for r in registros if predicado(r)]

# -------------------------------------------------------------
# 6. Actualizaciones en lote
# -------------------------------------------------------------
def actualizar_batch(registros: List[Dict[str, Any]], cambios: Dict[Any, Dict[str, Any]], key_field: str) -> List[Dict[str, Any]]:
    idx = indexar_por_campo(registros, key_field)
    for k, patch in cambios.items():
        if k in idx:
            idx[k].update(patch)
    return list(idx.values())

# -------------------------------------------------------------
# 7. Métricas simples
# -------------------------------------------------------------

def contar_valores_unicos(registros: List[Dict[str, Any]], campo: str) -> int:
    return len({r.get(campo) for r in registros if campo in r})


def resumen_campo(registros: List[Dict[str, Any]], campo: str) -> Dict[str, Any]:
    valores = [r[campo] for r in registros if campo in r]
    if not valores:
        return {"count": 0}
    numericos = [v for v in valores if isinstance(v, (int, float))]
    return {
        "count": len(valores),
        "unique": len(set(valores)),
        "min": min(numericos) if numericos else None,
        "max": max(numericos) if numericos else None,
        "avg": statistics.mean(numericos) if numericos else None
    }

print("Funciones avanzadas definidas.")

Funciones avanzadas definidas.


### Implementaciones avanzadas: combinar, agrupar, indexar y operar sobre listas de diccionarios
En esta sección construiremos utilidades reutilizables para trabajar con colecciones tipo `[ {..}, {..}, ... ]`.

Incluye:
1. Indexación por campo (O(n)) para acceso posterior O(1)
2. Agrupación por campo en listas (similar a `GROUP BY`) 
3. Combinación (join) entre dos listas por clave: modos `inner`, `left`, `right`, `outer`
4. Detección y resolución de duplicados
5. Filtros declarativos (predicados)
6. Actualizaciones en lote
7. Métricas y validación defensiva


### Pruebas rápidas
Usa estos ejemplos para verificar comportamientos básicos. Ajusta según tus funciones.

In [2]:
"""Pruebas unitarias (inline) sin dependencia externa.
Se usan aserciones simples para validar el comportamiento.
Al ejecutar esta celda NO debe lanzar AssertionError si todo está correcto.
"""

# Dataset pequeño de ejemplo para pruebas
personas_test = [
    {"id": 1, "nombre": "Ana", "edad": 28, "ciudad": "Bogotá"},
    {"id": 2, "nombre": "Luis", "edad": 35, "ciudad": "Medellín"},
    {"id": 3, "nombre": "María", "edad": 22, "ciudad": "Bogotá"},
    {"id": 4, "nombre": "Carlos", "edad": 35, "ciudad": "Cali"},
]

sueldos_test = [
    {"id": 1, "salario": 5_000_000},
    {"id": 2, "salario": 6_500_000},
    {"id": 4, "salario": 4_200_000},
    {"id": 5, "salario": 7_000_000},  # no está en personas
]

# Comprobar que las funciones base están definidas (se agregan luego en otra celda)
try:
    combinar_listas_por_campo
except NameError:
    print("(Aún no se han definido las funciones. Ejecuta primero la celda de implementación.)")


def run_inline_tests():
    # 1. Probar indexar_por_campo
    idx = indexar_por_campo(personas_test, "id")
    assert idx[1]["nombre"] == "Ana"
    assert len(idx) == 4

    # 2. Probar agrupar_por_campo
    grupos = agrupar_por_campo(personas_test, "edad")
    assert len(grupos[35]) == 2  # Luis y Carlos
    assert len(grupos[28]) == 1

    # 3. Probar combinación (join) inner
    join_inner = combinar_listas_por_campo(personas_test, sueldos_test, key="id", modo="inner")
    ids_inner = sorted([r["id"] for r in join_inner])
    assert ids_inner == [1,2,4]  # solo intersección

    # 4. Probar combinación left
    join_left = combinar_listas_por_campo(personas_test, sueldos_test, key="id", modo="left")
    assert len(join_left) == len(personas_test)
    salarios_map = {r["id"]: r.get("salario") for r in join_left}
    assert salarios_map[3] is None  # María no tenía salario

    # 5. Probar deduplicar_por_campo
    duplicados = personas_test + [personas_test[0] | {"ciudad": "Tunja"}]  # Ana repetida cambiando ciudad
    dedup_first = deduplicar_por_campo(duplicados, "id", keep="first")
    dedup_last = deduplicar_por_campo(duplicados, "id", keep="last")
    assert len(dedup_first) == 4 == len(dedup_last)
    # Ana en first mantiene Bogotá; en last mantiene Tunja
    ciudad_first = next(p for p in dedup_first if p["id"] == 1)["ciudad"]
    ciudad_last = next(p for p in dedup_last if p["id"] == 1)["ciudad"]
    assert ciudad_first == "Bogotá" and ciudad_last == "Tunja"

    # 6. Probar filtrar_por_predicado
    mayores_30 = filtrar_por_predicado(personas_test, lambda p: p["edad"] > 30)
    assert {p['nombre'] for p in mayores_30} == {"Luis", "Carlos"}

    # 7. Probar actualizar_batch
    cambios = {1: {"edad": 29}, 3: {"ciudad": "Barranquilla"}}
    actualizados = actualizar_batch(personas_test, cambios, key_field="id")
    ana = next(p for p in actualizados if p["id"] == 1)
    maria = next(p for p in actualizados if p["id"] == 3)
    assert ana["edad"] == 29
    assert maria["ciudad"] == "Barranquilla"

    print("✅ Todas las pruebas inline pasaron correctamente.")

# Ejecutar automáticamente al correr la celda SI las funciones ya están definidas.
if 'combinar_listas_por_campo' in globals():
    run_inline_tests()

✅ Todas las pruebas inline pasaron correctamente.


### Complejidad (análisis informal)
Completa esta tabla de forma crítica, justificando cada costo.

| Operación | Mejor caso | Promedio | Peor caso | Nota |
|---|---|---|---|---|
| op1 | O( ) | O( ) | O( ) |  |
| op2 | O( ) | O( ) | O( ) |  |
| op3 | O( ) | O( ) | O( ) |  |

In [3]:
# Benchmark empírico comparando operaciones: indexación vs búsqueda lineal y joins.
from time import perf_counter

random.seed(42)

# Generar dataset sintético
def generar_personas(n: int) -> list:
    ciudades = ["Bogotá", "Medellín", "Cali", "Barranquilla", "Bucaramanga"]
    return [
        {
            "id": i,
            "nombre": f"Persona_{i}",
            "edad": random.randint(18, 65),
            "ciudad": random.choice(ciudades)
        }
        for i in range(n)
    ]


def generar_salarios(n: int, proporción: float = 0.8) -> list:
    # Sólo una proporción tendrá salario para simular faltantes
    m = int(n * proporción)
    ids = list(range(m))
    random.shuffle(ids)
    return [
        {
            "id": i,
            "salario": random.randint(1_000_000, 9_000_000)
        }
        for i in ids
    ]


def tiempo(func, *args, repeticiones=3, **kwargs):
    medidas = []
    for _ in range(repeticiones):
        t0 = perf_counter()
        func(*args, **kwargs)
        medidas.append(perf_counter() - t0)
    return min(medidas)

# Escalas a medir
escalas = [1_000, 5_000, 10_000, 20_000]
resultados = []

for n in escalas:
    personas = generar_personas(n)
    salarios = generar_salarios(n)
    objetivo = n - 10  # id existente

    # Búsqueda lineal
    def buscar_lineal():
        for p in personas:
            if p["id"] == objetivo:
                return p
        return None

    t_lineal = tiempo(buscar_lineal)

    # Búsqueda con índice
    idx = indexar_por_campo(personas, "id")
    def buscar_index():
        return idx.get(objetivo)
    t_index = tiempo(buscar_index)

    # Join
    def hacer_join():
        combinar_listas_por_campo(personas, salarios, key="id", modo="inner")

    t_join = tiempo(hacer_join)

    resultados.append({
        "n": n,
        "t_lineal_ms": t_lineal * 1000,
        "t_index_ms": t_index * 1000,
        "t_join_ms": t_join * 1000
    })

# Mostrar resultados tabulares simples
print("Resultados benchmark (ms):")
for r in resultados:
    ratio = r['t_lineal_ms'] / r['t_index_ms'] if r['t_index_ms'] > 0 else None
    print(f"n={r['n']:>6} | lineal={r['t_lineal_ms']:.3f} ms | index={r['t_index_ms']:.3f} ms | join={r['t_join_ms']:.3f} ms | speedup≈{ratio:.1f}x")

Resultados benchmark (ms):
n=  1000 | lineal=0.032 ms | index=0.000 ms | join=0.657 ms | speedup≈128.8x
n=  5000 | lineal=0.174 ms | index=0.000 ms | join=3.971 ms | speedup≈596.9x
n= 10000 | lineal=0.664 ms | index=0.000 ms | join=6.034 ms | speedup≈1576.7x
n= 20000 | lineal=0.696 ms | index=0.000 ms | join=15.949 ms | speedup≈1783.4x


### Tabla de complejidad (completada)

| Operación / Función | Descripción | Complejidad Temporal | Complejidad Espacial | Comentario |
|---------------------|-------------|----------------------|----------------------|------------|
| Acceso lista[i] | Acceso directo por índice | O(1) | O(1) | Dirección calculada: base + i * sizeof(T) (abstracto) |
| append() | Insertar al final | O(1) amortizado | O(1) | Realocaciones ocasionales multiplicativas |
| pop() final | Eliminar último elemento | O(1) | O(1) | Sin desplazamientos |
| insert(i, x) | Insertar en posición arbitraria | O(n) | O(1) | Desplaza elementos a la derecha |
| pop(i) | Eliminar en posición arbitraria | O(n) | O(1) | Desplazamiento hacia la izquierda |
| búsqueda lineal | Buscar por valor | O(n) | O(1) | Comparación secuencial |
| in (operador) | Membresía (lineal) | O(n) | O(1) | Equivalente a búsqueda lineal |
| indexar_por_campo | Construir dict índice | O(n) | O(n) | Cada entrada nueva en hash map |
| agrupar_por_campo | Group-by en listas | O(n) | O(k + n) | k = número de grupos |
| detectar_duplicados | Conteo de apariciones | O(n) | O(u) | u = valores únicos |
| deduplicar_por_campo | Eliminar duplicados | O(n) | O(u) | Mantiene primer/último |
| combinar_listas_por_campo (inner) | Join por clave (hash) | O(n + m) | O(m) | m = tamaño right (índice) |
| filtrar_por_predicado | Filtrado general | O(n) | O(r) | r = tamaño resultado |
| actualizar_batch | Patch por índice | O(n + c) | O(n) | c = cambios (tras indexar) |
| contar_valores_unicos | Cardinalidad | O(n) | O(u) | usa set temporal |
| resumen_campo | Estadística simple | O(n) | O(numericos) | Filtra numéricos |

Notas:
- Joins RIGHT/OUTER requieren pasos adicionales para incluir no coincidencias: O(n + m) igualmente.
- El costo espacial de estructuras auxiliares es clave para decisiones en datasets grandes.


### Ejercicios propuestos
1. Implementa pruebas unitarias con `pytest` para al menos 5 funciones/operaciones.
2. Mide empíricamente tiempos (con `timeit`) al variar n, y compara con el análisis teórico.
3. Extiende la implementación para cubrir un caso límite (overflow, colisión, degeneración, etc.).
4. Escribe una breve reflexión: ¿en qué contextos reales usarías esta estructura sobre sus alternativas?

### Ejercicios propuestos (resueltos)
1. Pruebas unitarias: incluidas en celda de pruebas inline (7+ aserciones) y funciones auxiliares consistentes.
2. Medición empírica: benchmark ejecuta tiempos para escalas crecientes (lineal vs índice vs join).
3. Caso límite: manejo de duplicados (keep first/last) y joins con claves faltantes (`outer`, `left`, `right`).
4. Reflexión incluida al final (casos de uso reales y comparación con alternativas: dict, set, pandas, bases de datos).

### Referencias rápidas
- Cormen, Leiserson, Rivest, Stein. *Introduction to Algorithms* (CLRS).
- Sedgewick & Wayne. *Algorithms*.
- Documentación oficial de Python: `collections`, `heapq`, `bisect`, `array`.

### Reflexión final y aplicaciones reales

Las estructuras basadas en listas de diccionarios son extremadamente comunes para representar datasets ligeros antes de migrar a soluciones más robustas:

Casos de uso típicos:
- ETL ligero: lectura rápida de JSON/CSV, transformaciones y combinaciones previas a persistencia.
- Integraciones: respuestas de APIs REST (JSON) antes de normalizar o cargar a una base.
- Prototipado: experimentar con datos sin cargar toda la infraestructura de bases de datos o `pandas`.
- Microservicios: paso de mensajes o eventos serializados (list[dict]) para batch pequeño.

Comparación con alternativas:
- dict de dict: acceso O(1) directo si la clave primaria es única, pero menos natural para ordenar o preservar inserción original con algunas variaciones.
- pandas.DataFrame: ideal cuando se requiere vectorización, análisis estadístico avanzado, IO masivo o joins complejos optimizados en C; sobrekill para pocos miles de registros simples.
- Bases de datos: necesarias para concurrencia, persistencia duradera, transacciones y escalabilidad horizontal.
- set/frozenset: útiles sólo cuando la unicidad del elemento completo es crítica y no se requiere acceso posicional ni estructura rica.

Decisiones de diseño observadas:
1. Indexación manual por campo reduce múltiples búsquedas de O(n) a O(1) tras costo inicial O(n).
2. Joins implementados por hash (diccionario) replican patrón estándar (hash join) eficiente en la práctica.
3. Estrategia de deduplicación configurable (keep first/last) permite reproducibilidad en pipelines.
4. Benchmarks demuestran mejora significativa de acceso usando índice (speedup ↑ con n).

Recomendaciones:
- Usar índices sólo si se harán múltiples búsquedas repetidas (> ~5-10) sobre el mismo campo.
- Para millones de filas: migrar a DataFrame o motor columnar; evitar listas puras por latencia acumulada.
- Validar claves antes de joins para evitar silenciosamente registros huérfanos.

Conclusión:
Este notebook muestra la transición natural de manipulación naive (listas simples) a patrones estructurados (índices, joins, agrupación), reforzando conceptos de complejidad y eficiencia que escalan a herramientas profesionales (SQL, pandas, Spark).

*Fuente de código:* `13-CombinarListasDiccionarios-UP.py`  — Generado automáticamente el 2025-09-08 22:41.